## Extract trails
* This notebook will concatenate all epochs assoicated with a trail in to one data point.
* The goal of this is to try to describe data as a binary label
    * In other words a trail can be described as congruent or incongruent

In [ ]:
%run ../preprocessing/Artifact_Removal/preprocecssing_helpers.ipynb
%run ../preprocessing/StimCodes.ipynb
import os

import mne
from mne.preprocessing import ICA
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
matplotlib.rcParams.update({'font.size': 15})

from sklearn.preprocessing import StandardScaler, normalize

In [ ]:
path = 'E:\EpochedEEG'
epoch_files = os.listdir(path)
file = os.path.join(path,epoch_files[0])
epoch = mne.read_epochs(file, preload = True)

In [ ]:
#UPDATES THE STIM CODE EVENT_IDS TO THEIR CORRECT VALUES
epoch = update_stim_code_event_ids(epoch)

In [ ]:
epoch.event_id

In [ ]:
events = epoch.events[:,-1]
event_ids = epoch.event_id
print("len events:", len(events))

In [ ]:
epoch.get_data().shape

In [ ]:
#combining events of a trail into 1 list
trials = []
event_trial = []
start_index = 0
end_index = 0
trial_index_list = []
for event in events:
    if event == 31 and event_trial != []:
        trial_index_list.append((start_index, end_index))
        trials.append(event_trial)
        event_trial = []
        event_trial.append(event)
        start_index = i
    else:
        event_trial.append(event)
print(np.array(trials)[:5])

In [ ]:
#combining events of a trail into 1 list
trials_1 = []
event_trial_1 = []
start_index = 0
end_index = 0
trial_index_list = []
for i in range(len(events)):
    if events[i] == 31 and event_trial != []:
        trial_index_list.append((start_index, end_index))
        trials_1.append(event_trial_1)
        event_trial_1 = []
        event_trial_1.append(events[i])
        start_index = i
        end_index +=1
    else:
        end_index +=1
        event_trial_1.append(events[i])
print(np.array(trials_1)[:5])

In [ ]:
trial_index_list

In [ ]:
events[trial_index_list[-1][0]:trial_index_list[-1][1]]

In [ ]:
#create a list of dictionaries for trials where each dictionary
# contains the event_ids of a trial.
trials_and_events = []
trial_dict = {}
for trial in trials:
    for event in trial:
        for key, value in event_ids.items():
            if value == event:
                trial_dict[key] = event
    trials_and_events.append(trial_dict)
    trial_dict = {}
trials_and_events[:3]

In [ ]:
#remove dead trail at the beginning of study
# del trials_and_events[0]

###  Combine epochs by trail (event_ids are combined, but combining epochs is a different task)

### combine epochs and give them a label equal to Audio or Visual 

In [ ]:
# creating a new dictionary where each trail will equal one of the values in the dictinary Mode_lexi that is defined in 
# ../preprocessing/StimCodes.ipynb
trial_event_ids = []
for trial in trials_and_events:
    #check individual stim-codes and match them with corresponding combination defined in stimcodes.ipynb
    stim_combination = []
    
    for key in trial:
#         print(trial[key])
        if int(trial[key]) >= 1 and int(trial[key]) <= 24:
            stim_combination.append(trial[key])
#             print(stim_combination)
        if len(stim_combination) == 3: # stim-code combination complete, it can now be matched to a value in Mode_lexi
            stim_combination_tuple = (stim_combination[0],stim_combination[1],stim_combination[2])
#             print(stim_combination_tuple)
            
            for stim_combo_key in modal_lexi:
                stim_combo1 = modal_lexi[stim_combo_key][0]
                stim_combo2 = modal_lexi[stim_combo_key][1]

                if stim_combination_tuple == stim_combo1 or stim_combination == stim_combo2:
                    dict = {stim_combo_key : trial}      
                    trial_event_ids.append(dict)
            break;
 

In [ ]:
trial_event_ids[0] #this trial has stim-code combination VVLN, which is a visual stimuli 

## Event_ids are concatentated, but now I need to concatentate epochs
* This will be done by changing event_ids of a trial into a new event event_id that will describe all epochs in that trial. 

Trying a new method where I have one epoch object that will continuously be concatentated with another temporary epoch object.

In [ ]:
#new Event_ids
modality_lexicality_event_ids = {
   "AALL" : 112,
   "AALN" : 212,
   "AANL" : 312,
   "AANN" : 412,
   "AVLL" : 512,
   "AVNN" : 612,
   "VALL" : 712,
   "VANN" : 812,
   "VVLL" : 912,
   "VVLN" : 1012,
   "VVNL" : 1112,
   "VVNN" : 1212
}

Use mne.merge_events to combine events

In [ ]:
### temp_trial_list = [] #used for storage of event_ids of trial
temp_events = epoch.events
for trial in trial_event_ids[1:3]:
    for key in trial:
        for item in trial[key]:
            temp_trial_list.append(epoch.event_id[item])
        print(temp_trial_list)
        temp_events = mne.merge_events(temp_events, temp_trial_list, modality_lexicality_event_ids[key])
        temp_trial_list = []

In [ ]:
epoch.events[:,-1][:10]

In [ ]:

epoch_1 = epoch[1:9]
epochs_merge = mne.merge_events(events=epoch_1.events,ids = epoch_1.events[:,-1],new_id= 0 )

In [ ]:
epoch_1.events = epochs_merge
epoch_1.plot();

In [ ]:
start_time = epoch_1.events[:,0][0]
end_time = epoch_1.events[:,0][-1]
epoch_1.events = np.array([
    [2580,    0,   0],
    [2581,   31,  0],
    [2583,    0,  0],
    [2593,    0,    0],
    [2718,    0,    0],
    [2847,    0,  0],
    [3200,    0,  0],
    [3201,  132,  0],
])